<a href="https://colab.research.google.com/github/agnxsh/bert-baseline/blob/main/bert_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 78.7 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
text = "2023 will be the greatesr year for all of us!"
encoding = tokenizer.encode_plus(text, add_special_tokens = True, truncation = True, padding = "max_length",
                                return_attention_mask = True, return_tensors="pt")

#encode_plus returns a dictionary of values, other than just a list of values, because 
#encode_plus can return various types information, such as token_ids, attention_masks, etc, 
#hence the best thing to do in this case is to make it a dictionary, so that the individual 
#attributes are easily accessible

In [ ]:
encoding

{'input_ids': tensor([[  101, 16798,  2509,  2097,  2022,  1996,  2307,  2229,  2099,  2095,
          2005,  2035,  1997,  2149,   999,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
input = encoding["input_ids"][0]
attention_mask = encoding["attention_mask"][0]
attention_mask

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
tokenizer.mask_token

'[MASK]'

#Masked Language Modelling -> Basically fill in the blanks with the most appropriate word xD

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased", return_dict=True)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#logits are the prediction scores of the Language Modlling Head of the ouput of the BERT model, just before the softmax activation function,
#these logits are more like scoring system for ensuring which word in the vocabulary is the most appropriate.
text = " the Opera House in Australia is in, "+tokenizer.mask_token + " city"

In [ ]:
input = tokenizer.encode_plus(text, return_tensors="pt")

In [ ]:
mask_index = torch.where(input["input_ids"][0]==tokenizer.mask_token_id)

In [ ]:
output =  model(**input)
logits = output.logits

In [ ]:
softmax = F.softmax(logits, dim=-1)
#getting a probabilistic value
mask_word = softmax[0, mask_index, :]

In [ ]:
top_10 = torch.topk(mask_word, 10, dim=1)[1][0]
top_10.shape
top_10.dim

<function Tensor.dim>

In [ ]:
for token in top_10:
  word=  tokenizer.decode([token])
  new_sentence = text.replace(tokenizer.mask_token, word)
  print(new_sentence)

 the Opera House in Australia is in, sydney city
 the Opera House in Australia is in, melbourne city
 the Opera House in Australia is in, brisbane city
 the Opera House in Australia is in, adelaide city
 the Opera House in Australia is in, the city
 the Opera House in Australia is in, canberra city
 the Opera House in Australia is in, auckland city
 the Opera House in Australia is in, hobart city
 the Opera House in Australia is in, griffith city
 the Opera House in Australia is in, hume city


In [ ]:
#for the best result
softmax = F.softmax(logits, dim=-1)
mask_word = softmax[0,mask_index,:]
top_word = torch.argmax(mask_word,dim=1)
print(tokenizer.decode(top_word))

sydney


#Next Sentence Prediction

Next Sentence Prediction is the task of predicitng whether one sentence follows another sentence.
It essentially returns `(torch.FloatTensor of shape (batch_size,2))` - Prediction scores of the next sequence prediction.

In [16]:
from transformers import BertTokenizer, BertForNextSentencePrediction
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForNextSentencePrediction.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
prompt ="I came back from Office in the morning"
next_sentence ="I opened my Beer after Office"

In [19]:
encoding = tokenizer.encode_plus(prompt,next_sentence,return_tensors="pt")

In [20]:
outputs = model(**encoding)[0]
softmax = F.softmax(outputs, dim =1)

In [21]:
print(softmax)

tensor([[9.9999e-01, 1.4832e-05]], grad_fn=<SoftmaxBackward0>)


##Question Answering

In [26]:
from transformers import BertTokenizer, BertForQuestionAnswering
tokenizer = BertTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

In [27]:
example_text = "GPT-3 came in 2020"
example_question = "When did GPT-3 come?"

tokenized_inputs = tokenizer(example_question, example_text, return_tensors="pt")
tokenized_inputs

{'input_ids': tensor([[  101,  1332,  1225, 15175,  1942,   118,   124,  1435,   136,   102,
         15175,  1942,   118,   124,  1338,  1107, 12795,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}